# Импорт

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
import random
import datetime
from datetime import datetime
import warnings
import os, fnmatch
import yadisk
import shutil
import psycopg2
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 
import seaborn as sns
from tqdm import tqdm

warnings.simplefilter('ignore')
options = webdriver.ChromeOptions()
now = datetime.now()
options.headless = True



token = '*****'
dir_ = "*****"
EXE_PATH = r'C:\chromedriver.exe'

dates = {
    'января':'01',
    'февраля':'02',
    'марта':'03',
    'апреля':'04',
    'мая':'05',
    'июня':'06',
    'июля':'07',
    'августа':'08',
    'сентября':'09',
    'октября':'10',
    'ноября':'11',
    'декабря':'12'
}

host_2 = '****'
user_2 = '****'
password_2 = '****'
db_name_2 = '****'
port_2 = 5432

connection_2 = psycopg2.connect(
host=host_2,
    user=user_2,
    password=password_2,
    database = db_name_2,
    port=port_2
)


connection_2.autocommit = True

SQL_insert = "INSERT INTO arenda_ishevsk (type, adres, area, square, floor, build_type,\
furnish, price, m_price, date, url_link, ad_number, description)\
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

db_config = {'user': '*****', # имя пользователя
'pwd': '****', # пароль
'host': '****',
'port': ****, # порт подключения
'db': '****'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string)
y = yadisk.YaDisk(token=token)
print(y.check_token())


True


# Функции

In [18]:
def get_date_type(date):
    try:
        return datetime.strptime(date, "%d.%m.%Y")
    except:        
        return datetime.strptime(date[:10], "%Y-%m-%d")

def get_pages_number():
    pages = driver.find_elements("class name", "pagination-item-JJq_j")
    pages_ = []
    for page in pages:    
        if page.text.isdigit():        
            pages_.append(int(page.text))
    return(max(pages_))

def get_norm_links():
    all_links = []
    elems = driver.find_elements("xpath" ,"//a[@href]")
    for elem in elems:
        all_links.append(elem.get_attribute("href"))
    norm_links = []
    for link in all_links:
        if link.split('_')[-1].isdigit():
            norm_links.append(link)
    return(list(set(norm_links)))

def yandex():
    print('Начинаем перенос скринов на Я.диск')
    # Собираем файлы
    files = []
    listOfFiles = os.listdir('screenshots')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    # переносим в другую директорию
    for file in files:
        shutil.move('screenshots' +'\\' + file, os.getcwd())
    #записываем на Я.Диск
    c = 0        
    for file in files:
        print(f'Записываем {file}')
        if y.is_file(f'{dir_}{file}'):
            print('Такой уже есть')
            continue
        else:
            c+=1
            y.upload(file, f'{dir_}{file}')
    #удаляем файлы       
    files = []
    listOfFiles = os.listdir('.')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    for file in files:
        os.remove(file)
    print(f'Записано {c} файлов')

def get_adress():
    try:
        try:
            adress = driver.find_element("class name", "style-item-address__string-3Ct0s")
            return(adress.text)
        except:
            pass
        try:
            adress = driver.find_element("class name", "style-item-address__string-wt61A")
            return(adress.text)
        except:
            pass
    except:
        print('Ошибка в адресе')
        print(url)
        return(None)

def get_rayon():
    try:
       
        try:
            rayon = driver.find_element("class name", "style-item-address-georeferences-item-18pFt")
            return(rayon.text)
        except:
            pass
        
        try:
            rayon = driver.find_element("class name", "style-item-address-georeferences-item-TZsrp")
            return(rayon.text)
        except:
            pass
    
    except:        
        return(None)

def get_params():
    try:
        if len(driver.find_elements("class name", "params-paramsList__item-1Xeok")) > 0:
            params = driver.find_elements("class name", "params-paramsList__item-1Xeok")
            params_ = []
            for param in params:
                params_.append(param.text.split(':'))
            param_list = []
            for param in params_:
                  param_list.extend(param)
            return param_list
        
        if len(driver.find_elements("class name", "params-paramsList-zLpAu")) > 0:
            param_list = driver.find_elements("class name", "params-paramsList-zLpAu")[0].text.replace('\n', ' ').split(' ')
            return param_list
    except:
        print('Ошибка в параметрах')
        print(url)
        return('ERRROR')
    
def get_house_params():
    try:
        if len(driver.find_elements("class name", "style-item-params-list-3YJu7")) > 0:
            house_params = driver.find_elements("class name", "style-item-params-list-3YJu7")
            house_params_ = []

            for param in house_params:
                house_params_.append(param.text.replace('\n',' '))
            if len(house_params_) > 0:
                return(house_params_[0].split(' '))
            else:
                return('ERROR')
        if len(driver.find_elements("class name", "style-item-params-McqZq")) > 0:
            return driver.find_elements("class name", "style-item-params-McqZq")[0].text.replace('\n', ' ').split(' ')[2:]
    except:
        print('Ошибка в параметрах')
        print(url)
        return('ERRROR')
    
def get_square(kv_par):
    try:
        try:
            x = float(kv_par[kv_par.index('Общая площадь')+1].split()[0])
            return(x)
        except:
            pass
        try:
            x = float(kv_par[kv_par.index('площадь:')+1])
            return(x)
        except:
            pass
    except:        
        return(None)
    
def get_house_type(house_par): 
    try:
        try:
            x = house_par[house_par.index('здания:')+1]
            return(x)
        except:
            print('Ошибка в типе дома')
            print(url)
            return(None)
    except:       
        return(None)
    
def get_kv_floor(kv_par):
    try:
        try:
            x = int(kv_par[kv_par.index('Этаж:')+1].strip())
            return(x)
        except:
            pass
        try:
            x = kv_par[kv_par.index('Этаж')+1].strip()
            return(int(x.split()[0].strip()))
        except:
            pass
        try:
            x = kv_par[kv_par.index('Этаж:')+1].strip()
            return(x.split()[0].strip())
        except:
            pass
    except:       
        return(None)
    
def get_otdela(kv_par):
    try:
        try:
            x = kv_par[kv_par.index('Отделка')+1].strip()
            return(x)
        except:
            pass
        try:
            x = kv_par[kv_par.index('Отделка:')+1].strip()
            return(x)
        except:
            pass
    
        
    except:       
        return(None)
    
def get_price():
    try:
        test = driver.find_element("class name", "style-item-view-price-content-1aHxn")
    except:
        pass
    try:
        test = driver.find_element("class name", "style-item-price-PuQ0I")
    except:
        pass
    i = 0
    price = ''
    while True:
        if test.text[i].isdigit() or test.text[i] == ' ':
            price += test.text[i]
            i +=1
        elif i > len(test.text):
            break
        else:
            break
    try:
        if test.text.replace('\n', '').find('в месяц за м²') < 15 and test.text.replace('\n', '').find('в месяц за м²') > 0:
            price = float(price.replace(' ', '')) * get_square(kv_par)
            return(price)
        else:
            return float(price.replace(' ',''))
        
    except:        
        return(None)
    
def get_date():
    try:
        #1 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-1y5w6")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        # 2 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-EEthc")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        #3 ексепт
        try:
            test = driver.find_element("class name", "style-item-footer-Ufxh_")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
        
        try:
            test = driver.find_element("class name", "style-item-footer-3uXQz")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
        
        
    except:        
        return(None)
    
def is_bad_adv(i,j):
    try:
        driver.find_element("class name", "closed-warning-content-_f4_B").text
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        print(url)
        print('Снято с публикации')
        return(True)
    except:
        try:
            driver.find_element("class name", "desktop-16c9mq7").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print(url)
            print('Страницы не существует')
            return(True)
        except:        
            try:
                driver.find_element("class name", "closed-warning-content-2ooy4").text
                print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                print('Снято с публикации')
                print(url)
                return(True)
            except:
                try:
                    driver.find_element("class name", "page-title-count-wQ7pG").text
                    print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                    print(url)
                    print('Страница поиска, объявления нет')
                    return(True)
                except:
                        try:
                            driver.find_element("class name", "b-404").text
                            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                            print('Нет такой страницы')
                            print(url)
                            return(True)
                        except:
                            return(False)
def get_view():
    try:
        try:
            test = driver.find_element("class name", "style-item-description-text-mc3G6")
            return(test.text.replace('\n', ' '))
        except:
            pass
        
        try:
            test = driver.find_element("class name", "style-item-description-html-qCwUL")
            return(test.text.replace('\n', ' '))
        except:
            pass
        
        try:
            test = driver.find_element("class name", "style-item-description-html-1_RNo")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-SzN56")
            return(test.text.replace('\n', ' '))
        except:
            pass
    except:        
        return(None)
    
def get_type():
    try:
        try:
            test = driver.find_element("class name", "breadcrumbs-root-_GADZ")
            return test.text.split('·')[-1]
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-navigation-3dyI6")
            return test.text.split('·')[-1]
        except:
            pass
    except:
        return None

# Links

In [19]:
driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
driver.get('https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/sdam-ASgBAgICAUSwCNRW?p=1')
page_numbers = get_pages_number()
driver.close()
print(f'Количество страниц с ссылками: {page_numbers}')

links = []
for i in tqdm(range(1, page_numbers+1)):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(f'https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/sdam-ASgBAgICAUSwCNRW?p={i}')
    links.extend(get_norm_links())
    driver.close()
    time.sleep(random.randint(5,12))
links = set(links)

with connection_2.cursor() as cursor:
                    cursor.execute(
                    'TRUNCATE TABLE arenda_links'
                    )
        
query = "SELECT * FROM arenda_ishevsk"
old_links = list(pd.io.sql.read_sql(query, con = engine)['url_link'].values)

for link in links:
    if 'izhevsk' in link and link not in old_links:
        with connection_2.cursor() as cursor:
                    cursor.execute(
                    f'''INSERT INTO arenda_links (links) VALUES ('{link}')'''
                    )
    else:
        continue
        
query = "SELECT links FROM arenda_links"
print(f'Новых линков: {len(list(pd.io.sql.read_sql(query, con = engine)["links"].values))}')

Количество страниц с ссылками: 19


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [12:45<00:00, 40.28s/it]


Новых линков: 140


# Парсинг

In [20]:
# Обнуление счетчика
counts_ = '0'

with connection_2.cursor() as cursor:
    cursor.execute(
    f'''UPDATE arenda_count
    SET count = {counts_}'''
    )

In [21]:
query = "SELECT * FROM arenda_count"
i = int(pd.io.sql.read_sql(query, con = engine)['count'].values[0])
i

0

## Сам парсер

In [22]:
query = "SELECT * FROM arenda_links"
test = pd.io.sql.read_sql(query, con = engine)
links = list(test['links'].values)


query = "SELECT * FROM arenda_count"
i = int(pd.io.sql.read_sql(query, con = engine)['count'].values[0])
#data = pd.read_csv('data.csv', index_col=0)
# Пустой ДФ
options.headless = True
df = {}
df['Тип помещения'] = [] 
df['Адрес'] = [] 
df['Район'] = []  
df['Общая площадь'] = [] 
df['Этаж'] = [] 
df['Тип здания'] = []
df['Отделка'] = []
df['Стоимость'] = [] 
df['Метр квадратный'] = [] 
df['Дата объявления'] = []
df['Ссылка'] = [] 
df['номер объявления'] = [] 
df['Описание'] = [] 

for j,url in enumerate (links[i:]):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(url)
    
    
    if j % 5 == 0 and j > 1:        
        df = pd.DataFrame(df)
        for q in range (len(df)):
            values_ = []
            for value in df.loc[q].values:
                values_.append(str(value))
            with connection_2.cursor() as cursor:
                    cursor.execute(
                    SQL_insert, [*values_]
                    )
        
        counts_ = str(i+j)
        with connection_2.cursor() as cursor:
            cursor.execute(
            f'''UPDATE arenda_count
            SET count = {counts_}'''
            )
        
        df = {}
        df['Тип помещения'] = [] 
        df['Адрес'] = [] 
        df['Район'] = []  
        df['Общая площадь'] = [] 
        df['Этаж'] = [] 
        df['Тип здания'] = []
        df['Отделка'] = []
        df['Стоимость'] = [] 
        df['Метр квадратный'] = [] 
        df['Дата объявления'] = []
        df['Ссылка'] = [] 
        df['номер объявления'] = [] 
        df['Описание'] = []
        
        print(f'Данные записаны на {i+j} позиции')
        print()
        
        yandex()
        
    if is_bad_adv(i,j): 
        driver.close()
        time.sleep(2)
        continue    
    try:            
        kv_par = get_params()
        house_par = get_house_params()
        #print('---'*22)
        df['Тип помещения'].append(get_type())
        df['Адрес'].append(get_adress())
        df['Район'].append(get_rayon())
        df['Общая площадь'].append(get_square(kv_par))
        df['Этаж'].append(get_kv_floor(kv_par))
        df['Тип здания'].append(get_house_type(house_par))
        df['Отделка'].append(get_otdela(kv_par))
        df['Стоимость'].append(get_price())
        
        df['Дата объявления'].append(get_date())
        df['Ссылка'].append(url)
        df['номер объявления'].append(df['Ссылка'][-1].split('_')[-1])
        df['Описание'].append(get_view())
        try:
            df['Метр квадратный'].append(round(df['Стоимость'][-1]/df['Общая площадь'][-1],2))
        except:
            print('Ошибка в определении стоимости метра кв.')
            df['Метр квадратный'].append(None)
                
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        if df["Адрес"][-1] == None:
            print(f'Адрес: {df["Адрес"][-1]}')
        if df["Тип помещения"][-1] == None:
            print(f'Тип помещения: {df["Тип помещения"][-1]}')
        if df["Район"][-1] == None:
            print(f'Район: {df["Район"][-1]}')
        if df["Общая площадь"][-1] == None:
            print(f'Общая площадь: {df["Общая площадь"][-1]}')
        if df["Этаж"][-1] == None:
            print(f'Этаж: {df["Этаж"][-1]}')        
        if df["Стоимость"][-1] == None:
            print(f'Стоимость: {df["Стоимость"][-1]}')
        if df["Дата объявления"][-1] == None:
            print(f'Дата объявления: {df["Дата объявления"][-1]}')
        print(f'Ссылка: {df["Ссылка"][-1]}')
        if df["Отделка"][-1] == None:
            print(f'Отделка: {df["Отделка"][-1]}')
        if df["Тип здания"][-1] == None:
            print(f'Тип здания: {df["Тип здания"][-1]}')
        if df["Описание"][-1] == None:
            print(f'Описание: {df["Описание"][-1]}')
                        
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element('tag name' ,'body').screenshot(f'screenshots/{url.split("_")[-1]}.png')
        print('Скриншот сделан')
        driver.quit()
        print(f'Стоимость: {df["Стоимость"][-1]}')
        print(f'Площадь: {df["Общая площадь"][-1]}')
        print(f'Стоимость кв.м. : {df["Метр квадратный"][-1]}')
        print()
        
        #print('---'*22)
        

    except:
        print('Тут что-то пошло не так (ошибка с вылетом скрипта):')
        print(url)        
        break
        
        #j +=1
    
    time.sleep(random.randint(10,19))

else:
    df = pd.DataFrame(df)
    for q in range (len(df)):
        values_ = []
        for value in df.loc[q].values:
            values_.append(str(value))
        with connection_2.cursor() as cursor:
                cursor.execute(
                SQL_insert, [*values_]
                )

    yandex()
    print('###'*22)
    print('#'+' '*64+'#')
    print('#'+ ' ' *20 + 'СКРИПТ ОКОНЧИЛ РАБОТУ!' + ' '*22 + '#')
    print('#'+' '*64+'#')
    print('###'*22)

****************************************** № 0 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/holodilnaya_kamera_60_m_2330248151
Скриншот сделан
Стоимость: 22000.0
Площадь: 60.0
Стоимость кв.м. : 366.67

****************************************** № 1 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_88_m_547782140
Скриншот сделан
Стоимость: 48000.0
Площадь: 88.0
Стоимость кв.м. : 545.45

****************************************** № 2 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/ofis_169.9_m_2561390539
Скриншот сделан
Стоимость: 76000.0
Площадь: 169.9
Стоимость кв.м. : 447.32

****************************************** № 3 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/pervaya_liniya_153_m.vysokiy_avt

Скриншот сделан
Стоимость: 49400.0
Площадь: 52.0
Стоимость кв.м. : 950.0

****************************************** № 26 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_naznacheniya_65.7_m_2510064886
Скриншот сделан
Стоимость: 65700.0
Площадь: 65.7
Стоимость кв.м. : 1000.0

****************************************** № 27 из 140 ******************************************
Этаж: None
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/zdanie_230_m_2461375989
Скриншот сделан
Стоимость: 138000.0
Площадь: 230.0
Стоимость кв.м. : 600.0

****************************************** № 28 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/obschepit_40_m_2435446168
Скриншот сделан
Стоимость: 60000.0
Площадь: 40.0
Стоимость кв.м. : 1500.0

****************************************** № 29 из 140 ******************************************
Ссылка: ht

Скриншот сделан
Стоимость: 180000.0
Площадь: 180.0
Стоимость кв.м. : 1000.0

****************************************** № 51 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/sklad_1061_m_1863252785
Отделка: None
Скриншот сделан
Стоимость: 957.0
Площадь: 1061.0
Стоимость кв.м. : 0.9

****************************************** № 52 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/studiya_30_m_2536106268
Скриншот сделан
Стоимость: 11000.0
Площадь: 30.0
Стоимость кв.м. : 366.67

****************************************** № 53 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/ofis_46_m_2553506662
Скриншот сделан
Стоимость: 29900.0
Площадь: 46.0
Стоимость кв.м. : 650.0

****************************************** № 54 из 140 ******************************************
Ссылка: https://www.avito.ru/izh

****************************************** № 76 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/sdam_2_ofisa_20_i_26_m_2492223954
Скриншот сделан
Стоимость: 20700.0
Площадь: 46.0
Стоимость кв.м. : 450.0

****************************************** № 77 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_naznacheniya_14_m_2300853519
Скриншот сделан
Стоимость: 8000.0
Площадь: 11.0
Стоимость кв.м. : 727.27

****************************************** № 78 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/sdam_chast_pomescheniya_15_m2_2528239771
Скриншот сделан
Стоимость: 6000.0
Площадь: 15.0
Стоимость кв.м. : 400.0

****************************************** № 79 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_n

Скриншот сделан
Стоимость: 147400.0
Площадь: 134.0
Стоимость кв.м. : 1100.0

****************************************** № 101 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/ofis_44_m_2127154223
Скриншот сделан
Стоимость: 30800.0
Площадь: 44.0
Стоимость кв.м. : 700.0

****************************************** № 102 из 140 ******************************************
Этаж: None
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/zdanie_550_m_2415682335
Скриншот сделан
Стоимость: 900.0
Площадь: 550.0
Стоимость кв.м. : 1.64

****************************************** № 103 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_naznacheniya_19.6_m_2553812184
Скриншот сделан
Стоимость: 12000.0
Площадь: 19.6
Стоимость кв.м. : 612.24

****************************************** № 104 из 140 ******************************************
Ссылка: https

Скриншот сделан
Стоимость: 45000.0
Площадь: 60.0
Стоимость кв.м. : 750.0

****************************************** № 126 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/torgovaya_ploschad_102.5_m_2569143656
Скриншот сделан
Стоимость: 123000.0
Площадь: 102.5
Стоимость кв.м. : 1200.0

****************************************** № 127 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/sdam_160_270_430_m_1_liniya_otd._vhod_2459994254
Скриншот сделан
Стоимость: 128000.0
Площадь: 430.0
Стоимость кв.м. : 297.67

****************************************** № 128 из 140 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/ofis_12_m_2036397237
Скриншот сделан
Стоимость: 9000.0
Площадь: 12.0
Стоимость кв.м. : 750.0

****************************************** № 129 из 140 ******************************************


# Предобработка

In [23]:
query = "SELECT * FROM arenda_ishevsk"
df = pd.io.sql.read_sql(query, con = engine)
df = df[df.columns[1:]]
df.head()

,type,adres,area,square,floor,build_type,furnish,price,m_price,date,url_link,ad_number,description
0,Помещение свободного назначения,"Удмуртская Республика, Ижевск, ул. Карла Маркс...",р-н Октябрьский,88.0,1,жилой,без,88000.0,1000.0,21.9.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2396211168,Сдам 88 кв.м. в новом доме Жк Эколайф! 1 линия...
1,Помещение свободного назначения,"Удмуртская Республика, Ижевск, ул. Михаила Пет...",р-н Устиновский,651.3,подвальный,жилой,без,403806.0,620.0,12.09.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2241186282,В аренду сдаются офисные помещения и помещения...
2,Помещение свободного назначения,"Удмуртская Республика, Ижевск, ул. Максима Гор...",р-н Октябрьский,192.0,1,жилой,офисная,200000.0,1041.67,29.08.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2542380192,"Сдается в аренду помещение в ЖК ""Колизей"", по ..."
3,Офисное помещение,"Удмуртская Республика, Ижевск, ул. Ленина, 100Б",р-н Первомайский,71.9,2,административное,чистовая,16362.0,227.57,9.09.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2440051558,"Сдается просторный офис, 60,6 кв.м., 2 этаж, д..."
4,Помещение свободного назначения,"Удмуртская Республика, Ижевск, Ленинский район...",р-н Ленинский,10.0,1,жилой,чистовая,6000.0,600.0,13.09.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2465780572,"Кабинет под услуги маникюра/педикюра, находящи..."


In [24]:
df.duplicated(subset=['ad_number']).sum()

6

In [25]:
df = df.drop_duplicates(subset=['ad_number'])
cols = df.columns
for col in cols:
    print(f'{col}: {df[col].isna().sum()}')

type: 0
adres: 0
area: 0
square: 0
floor: 0
build_type: 0
furnish: 0
price: 0
m_price: 0
date: 0
url_link: 0
ad_number: 0
description: 0


In [35]:
for col in cols:
    print(f'{col}: {(df[col] == "nan").sum()}')

type: 0
adres: 0
area: 0
square: 0
floor: 0
build_type: 0
furnish: 0
price: 0
m_price: 0
date: 0
url_link: 0
ad_number: 0
description: 0


In [32]:
for col in cols:
    print(f'{col}: {(df[col] == "None").sum()}')

type: 0
adres: 0
area: 0
square: 0
floor: 0
build_type: 0
furnish: 38
price: 0
m_price: 0
date: 0
url_link: 0
ad_number: 0
description: 0


In [34]:
df = df[df['square'] != 'nan']
df = df[df['type'] != 'None']
df = df[df['area'] != 'None']
df = df[df['floor'] != 'None']
df = df[df['floor'] != 'nan']

In [36]:
df['square'] = df['square'].apply(float)
df['price'] = df['price'].apply(float)
df['m_price'] = df['m_price'].apply(float)
df['datetime'] = df['date'].apply(get_date_type)
df = df[df['m_price'] >= 20]

In [37]:
df.shape

(918, 14)

In [38]:
df.to_excel('arenda_izhevsk.xlsx', index=0)